# Spark Metastore

Let us understand how to interact with metastore tables using Spark based APIs.

* Overview of Spark Metastore
* Starting Spark Context
* Spark Catalog
* Creating Metastore Tables
* Define Schema for Tables
* Inserting into Existing Tables

## Overview of Spark Metastore

Let us get an overview of Spark Metastore and how we can leverage it to manage databases and tables on top of Big Data based file systems such as HDFS, s3 etc.

* Quite often we need to deal with structured data and the most popular way of processing structured data is by using Databases, Tables and then SQL.
* Spark Metastore (similar to Hive Metastore) will facilitate us to manage databases and tables.
* Typically Metastore is setup using traditional relational database technologies such as **Oracle**, **MySQL**, **Postgres** etc.

## Starting Spark Context

Let us start spark context for this Notebook so that we can execute the code provided.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    appName('Spark Metastore'). \
    master('yarn'). \
    getOrCreate()

In [ ]:
spark.conf.set('spark.sql.shuffle.partitions', '2')

## Spark Catalog

Let us get an overview of Spark Catalog. It is part of `SparkSession` object.
* We can permanently or temporarily create tables or views on top of data in a Data Frame.
* Metadata such as table names, column names, data types etc for these tables or views will be stored in Metastore. It is also known as catalog which is exposed as part of SparkSession object.
* Permanent tables can be grouped into databases in metastore. If not specified, the tables will be created in **default** database.
* Let us say `spark` is of type `SparkSession`. There is an attribute as part of `spark` called as catalog and it is of type pyspark.sql.catalog.Catalog.
* We can access catalog using `spark.catalog`.
* There are several methods that is part of `spark.catalog`. We will explore them in the later topics.
* Following are some of the tasks that can be performed using `spark.catalog` object.
 * Check current database and switch to different databases.
 * Create permanent table in metastore.
 * Create or drop temporary views.
 * Register functions.
* All the above tasks can be performed using SQL style commands passed to `spark.sql`.

## Creating Metastore Tables

Data Frames can be written into Metastore Tables using APIs such as `saveAsTable` and `insertInto` available as part of write on top of objects of type Data Frame.

* We can create a new table using Data Frame using `saveAsTable`. We can also create an empty table by using `spark.catalog.createTable` or `spark.catalog.createExternalTable`.
* We can also prefix the database name to write data into tables belong to a particular database. If the database is not specified then the session will be attached to default database.
* Databases can be created using `spark.sql("CREATE DATABASE database_name")`. We can list Databases using `spark.sql` or `spark.catalog.listDatabases()`
* We can use modes such as `append`, `overwrite` and `error` with `saveAsTable`. Default is error.
* We can use modes such as `append` and `overwrite` with `insertInto`. Default is append.
* When we use `saveAsTable`, following happens:
 * Check for table if the table already exists. By default `saveAsTable` will throw exception.
 * If the table does not exists the table will be created.
 * Data from Data Frame will be copied into the table.
 * We can alter the behavior by using mode. We can overwrite the existing table or we can append into it.
* We can list the tables using `spark.catalog.listTables` after switching to appropriate database using `spark.catalog.setCurrentDatabase`.
* We can also switch the database and list tables using `spark.sql`.

In [ ]:
spark.catalog?

### Tasks
Let us perform few tasks to understand how to write a Data Frame into Metastore tables and also list them.
* Create database by name db in the metastore. We need to use `spark.sql` as there is no function to create database under `spark.catalog`.

In [ ]:
import getpass
username = getpass.getuser()

In [ ]:
spark.sql(f"CREATE DATABASE {username}_db")

In [ ]:
spark.catalog.setCurrentDatabase(f'{username}_db')

* List the databases using both API as well as SQL approach. As we have too many databases in our environment, it might take too much time to return the results

In [ ]:
spark.catalog.listDatabases()

* Create a Data Frame which contain one column by name **dummy** and one row with value **X**.

In [ ]:
l = [("X", )]
df = spark.createDataFrame(l, schema="dummy STRING")

In [ ]:
spark.catalog.listTables()

* Create a table by name dual for the above Data Frame in the database created. 

In [ ]:
df.write.saveAsTable("dual")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.read.table("dual").show()

* Let us drop the table **dual** and then database **db**. We need to use `spark.sql` as `spark.catalog` does not have API to drop the tables or databases.

In [ ]:
spark.sql("DROP TABLE dual")

In [ ]:
spark.sql(f"DROP DATABASE {username}_db")

In [ ]:
# We can use CASCADE to drop database along with tables.
spark.sql(f"DROP DATABASE {username}_db CASCADE")

## Define Schema for Tables

When we want to create a table using `spark.catalog.createTable` or using `spark.catalog.createExternalTable`, we need to specify Schema.

* Schema can be inferred or we can pass schema using `StructType` object while creating the table..
* `StructType` takes list of objects of type `StructField`.
* `StructField` is built using column name and data type. All the data types are available under `pyspark.sql.types`.
* We need to pass table name and schema for `spark.catalog.createTable`.
* We have to pass path along with name and schema for `spark.catalog.createExternalTable`.
* We can use source to define file format along with applicable options. For example, if we want to create a table for CSV, then source will be csv and we can pass applicable options for CSV such as sep, header etc.

### Tasks

Let us perform tasks to create empty table using `spark.catalog.createTable` or using `spark.catalog.createExternalTable`.

* Create database **hr_db** and table **employees** with following fields. Let us create Database first and then we will see how to create table.
 * employee_id of type Integer
 * first_name of type String
 * last_name of type String
 * salary of type Float
 * nationality of type String

In [ ]:
import getpass
username = getpass.getuser()

In [ ]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {username}_hr_db")

In [ ]:
spark.catalog.setCurrentDatabase(f"{username}_hr_db")

In [ ]:
spark.catalog.createTable?

* Build StructType object using StructField list.

In [ ]:
from pyspark.sql.types import StructField, StructType, \
    IntegerType, StringType, FloatType

In [ ]:
employeesSchema = StructType([
    StructField("employee_id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("salary", FloatType()),
    StructField("nationality", StringType())
])

In [ ]:
spark.sql('DROP TABLE employees')

* Create table by passing StructType object as schema.

In [ ]:
spark.catalog.createTable("employees", schema=employeesSchema)

* List the tables from database created.

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.catalog.listColumns('employees')

In [ ]:
spark.sql('DESCRIBE FORMATTED employees').show(100, truncate=False)

* Create database by name **{username}_airlines** and create external table for **airport-codes.txt**.
 * Data have header
 * Fields in each record are delimited by a tab character.
 * We can pass options such as sep, header, inferSchema etc to define the schema.


In [ ]:
spark.catalog.createExternalTable?

In [ ]:
import getpass
username = getpass.getuser()

In [ ]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {username}_airlines")

In [ ]:
spark.catalog.setCurrentDatabase(f"{username}_airlines")

* To create external table, we need to have write permissions over the path which we want to use.
* As we have only read permissions on **/public/airlines_all/airport-codes** we cannot use that path while creating external table.
* Let us copy the data to **/user/`whoami`/airlines_all/airport-codes**

In [ ]:
%%sh

hdfs dfs -mkdir -p /user/`whoami`/airlines_all
hdfs dfs -cp -f /public/airlines_all/airport-codes /user/`whoami`/airlines_all
hdfs dfs -ls /user/`whoami`/airlines_all/airport-codes

In [ ]:
%%sh

hdfs dfs -tail /user/`whoami`/airlines_all/airport-codes/airport-codes-na.txt

In [ ]:
import getpass
username = getpass.getuser()

airport_codes_path = f'/user/{username}/airlines_all/airport-codes'

In [ ]:
spark.sql('DROP TABLE airport_codes')

In [ ]:
spark.catalog. \
    createExternalTable("airport_codes",
                        path=airport_codes_path,
                        source="csv",
                        sep="\t",
                        header="true",
                        inferSchema="true"
                       )

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.read.table("airport_codes").show()

In [ ]:
spark.sql('DESCRIBE FORMATTED airport_codes').show(100, False)

## Inserting into Existing Tables

Let us understand how we can insert data into existing tables using `insertInto`.

* We can use modes such as `append` and `overwrite` with `insertInto`. Default is `append`.
* When we use `insertInto`, following happens:
 * If the table does not exist, `insertInto` will throw an exception.
 * If the table exists, by default data will be appended.
 * We can alter the behavior by using keyword argument overwrite. It is by default False, we can pass True to replace existing data.

### Tasks

Let us perform few tasks to understand how to write a Data Frame into existing tables in the Metastore.

* Make sure hr_db database and employees table in hr_db are created.

In [ ]:
spark.catalog.listDatabases()

In [ ]:
spark.catalog.setCurrentDatabase(f"{username}_hr_db")

In [ ]:
spark.catalog.listTables()

Use employees Data Frame and insert data into the employees table in hr_db database. Make sure existing data is overwritten.

In [ ]:
employees = [(1, "Scott", "Tiger", 1000.0, "united states"),
             (2, "Henry", "Ford", 1250.0, "India"),
             (3, "Nick", "Junior", 750.0, "united KINGDOM"),
             (4, "Bill", "Gomes", 1500.0, "AUSTRALIA")
            ]

In [ ]:
employeesDF = spark.createDataFrame(employees,
    schema="""employee_id INT, first_name STRING, last_name STRING,
              salary FLOAT, nationality STRING
           """
)

In [ ]:
employeesDF.write.insertInto("employees", overwrite=True)

In [ ]:
employeesDF.write.mode('overwrite').insertInto("employees")

In [ ]:
spark.read.table("employees").show()